# Using JDBC Reader in your notebooks

This notebook uses the code from the [medium article](https://medium.com/@orellabac/ingest-external-data-into-snowflake-with-snowpark-and-jdbc-eb487b61078c)

It explains for example how to get it working in your notebook

For making it easier we have provided a [simple file](https://raw.githubusercontent.com/Snowflake-Labs/snowpark-extensions-py/main/extras/jdbc_read/jdbc_reader.py) you can use to load this "extension" and
register the UDF.

> You need an external access integration and a network rule previously created so you can register the UDF. You can see the [documentation for more details](https://docs.snowflake.com/en/developer-guide/external-network-access/creating-using-external-network-access).



An easy setup is to setup an stage for example `JDBC_DRIVERS` 
![jdbc_drivers_stage](https://raw.githubusercontent.com/Snowflake-Labs/snowpark-extensions-py/main/extras/jdbc_read/jdbc_driver.png)

After you have setup this stage add a reference to the `jdbc_reader.py` 
![](https://github.com/Snowflake-Labs/snowpark-extensions-py/blob/main/extras/jdbc_read/packages.png?raw=true)

Ok once you have done that we can import the `jdbc_reader` module and use the 
`jdbc_reader.register_jdbc_reader` function.

We just need to pass the name of the STAGE where we have all our jars and the name of the integration. You can also pass a name to a secret which is more secure way to store credentials.


In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

import jdbc_reader
jdbc_reader.register_jdbc_reader("JDBC_DRIVERS","ALLOW_ALL_EAI")

After registering the function you can now used the python extensions to read from an external JDBC source. For example from a Azure SQL Database

In [ ]:
from snowflake.snowpark.functions import object_keys,sql_expr,typeof
url="jdbc:sqlserver://snowpark-sqlserver-tests.database.windows.net:1433;database=external-access-database"

data = session.read.format("jdbc")\
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
.option("url", url)\
.option("username", "admin@snowpark-sqlserver-tests")\
.option("password", "xxxxx")\
.query("""
SELECT AddressID, AddressLine1, AddressLine2, City
FROM [external-access-database].SalesLT.Address""").load().cache_result()

data